In [1]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
import time

이 노트북은 https://bright-ocean.tistory.com/79 블로그의 코드를 조금더 stable하게 수정한 것이다.

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [16]:
# 호주 주식시장 정보 페이지 접속
driver = webdriver.Chrome('./chromedriver.exe')
first_url = 'https://au.investing.com/equities/australia'
driver.get(first_url)

In [4]:
#select all australia ordinaries
select_box = driver.find_element_by_xpath('//*[@id="stocksFilter"]').click()
select_all_stock = driver.find_element_by_xpath('//*[@id="all"]').click()
time.sleep(8)

In [5]:
# 인베스팅 닷컴은 동적페이지이다. 
# BeautifulSoup을 사용할수 있도록 스크롤을 한번 끝까지 내렸다 올린다.
SCROLL_PAUSE_SEC = 3

# 스크롤 높이 가져옴
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # 끝까지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 1초 대기
    time.sleep(SCROLL_PAUSE_SEC)

    # 스크롤 다운 후 스크롤 높이 다시 가져옴
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [6]:
soup = BeautifulSoup(driver.page_source,"html.parser")

# 주식 테이블을 가지고 온다. 
maintable = soup.find('table', id='cross_rate_markets_stocks_1')


In [17]:
# 메인 주식 테이블에서 회사 페이지로
all_atag_maintable = maintable.find_all('a')
company_links = []

for a in all_atag_maintable:
    company_link = a.attrs["href"] #find로 한번더 돌려준다
    company_links.append(company_link)

In [18]:
base_url = 'https://au.investing.com'

IS_suffix = '-income-statement'
BS_suffix = '-balance-sheet'
CF_suffix = '-cash-flow'

In [19]:
company = company_links[0]
driver.get('https://au.investing.com'+company+IS_suffix)

In [28]:
#cross_rate_markets_stocks_1

result_df = pd.DataFrame()
for company in company_links[0:1] : 
    try : 
        #income data
        driver.get('https://au.investing.com'+company+IS_suffix)
        time.sleep(2)
        driver.find_element_by_xpath('/html/body/div[5]/section/div[8]/div[1]/a[1]').click()
        time.sleep(2)
        #driver.find_element_by_xpath('//*[id="excol"]').click()
        time.sleep(2)
        df_income = pd.read_html(driver.page_source)[0].dropna()
        time.sleep(2)
     
        """
        # balance data
        driver.get('https://au.investing.com'+company+BS_suffix)
        time.sleep(2)
        driver.find_element_by_xpath('//*[@id="leftColumn"]/div[8]/div[1]/a[1]').click()
        time.sleep(2)
        driver.find_element_by_xpath('//*[id="excol"]').click()
        time.sleep(2)
        df_balance = pd.read_html(driver.page_source)[0].dropna()
        time.sleep(2)

        #cash flow
        driver.get('https://au.investing.com'+company+CF_suffix)
        time.sleep(2)
        driver.find_element_by_xpath('//*[@id="leftColumn"]/div[8]/div[1]/a[1]').click()
        time.sleep(2)
        driver.find_element_by_xpath('//*[id="excol"]').click()
        time.sleep(2)
        df_cash_flow = pd.read_html(driver.page_source)[0].dropna()
        df_cash_flow.columns = df_cash_flow.columns.droplevel(level = 1)
        time.sleep(2)


        #dataframe
        df = df_income.append(df_balance, ignore_index=True)
        df = df.appennd(df_cash_flow, ignore_index = True)
        df = df.set_index('Period Ending:')
        df.index.name = None
        df.columns = ['2021','2020']
        stacked_df = pd.DataFrame(df.stack())
        index = stacked_df.index.to_flat_index()
        stacked_df.index = index
        stacked_df = stacked_df.T
        company_name = driver.find_element_by_xpath('//*[id="leftColumn"]/div[1]/h1').text
        name_df = pd.DataFrame([{'Company' : company_name}])
        company_df = pd.concat([name_df, stacked_df], axis = 1)
        result_df = result_df.append(company_df, ignore_index = True)
        """
        #close company info page?
        driver.close

    except : 
        print('fuck')

In [29]:
df_income

,Period Ending:,202131/12,202031/12,201931/12,201831/12
0,Total Revenue,-,0.01,0.02,0.02
1,"Revenue - 0.01 0.02 0.02 Other Revenue, Total...","Revenue - 0.01 0.02 0.02 Other Revenue, Total...","Revenue - 0.01 0.02 0.02 Other Revenue, Total...","Revenue - 0.01 0.02 0.02 Other Revenue, Total...","Revenue - 0.01 0.02 0.02 Other Revenue, Total..."
2,Revenue,-,0.01,0.02,0.02
3,"Other Revenue, Total",-,-,-,-
4,"Cost of Revenue, Total",1.96,1.84,1.98,1.88
5,Gross Profit,-1.96,-1.84,-1.96,-1.86
6,Total Operating Expenses,5.97,19.74,33.66,3.67
7,"Selling/General/Admin. Expenses, Total 3.87 1....","Selling/General/Admin. Expenses, Total 3.87 1....","Selling/General/Admin. Expenses, Total 3.87 1....","Selling/General/Admin. Expenses, Total 3.87 1....","Selling/General/Admin. Expenses, Total 3.87 1...."
8,"Selling/General/Admin. Expenses, Total",3.87,1.58,1.35,1.59
9,Research & Development,-,-,-,-


In [ ]:
#cross_rate_markets_stocks_1

#click all companies to get data
companies_table = driver.find_element_by_xpath('//*[@id="cross_rate_markets_stocks_1"]/tbody')
time.sleep(2)
tr_list = companies_table.find_elements_by_tag_name('tr')
time.sleep(2)
result_df = pd.DataFrame()
for tr in tr_list : 
    try : 
        link = tr.find_element_by_css_selector('td.bold.left.noWrap.elp.plusIconTd')
        link = link.find_element_by_tag_name('a')

        #p[em tje company info page on new brower
        link.send_keys(Keys.CONTROL + '\n')
        driver.switch_to.window(driver.window_handles[1])

        # goes to finance section
        finance_section = driver.find_element_by_xpath('//*[@id="pairSublinksLevel1"/li[4]/a').click()
        time.sleep(2)

        #income data
        income = driver.find_element_by_xpath('//*[@id=pairSublinksLevel2"]').click()
        time.sleep(2)
        annual_income = driver.find_element_by_xpath('//*[@id="leftColumn"]/div[8]/div[1]/a[1]').click()
        time.sleep(2)
        income_table = driver.find_element_by_xpath('//*[id="excol"]').click()
        time.sleep(2)
        df_income = pd.read_html(driver.page_source)[0].dropna()
        time.sleep(2)

        # balance data
        balance = driver.find_element_by_xpath('//*[@id=pairSublinksLevel2"]/li[3]/a').click()
        time.sleep(2)
        annual_balnace = driver.find_element_by_xpath('//*[@id="leftColumn"]/div[1]/a[1]').click()
        time.sleep(2)
        balance_table = driver.find_element_by_xpath('//*[id="excol"]').click()
        time.sleep(2)
        df_balance = pd.read_html(driver.page_source)[0].dropna()
        time.sleep(2)

        #cash flow
        cash_flow = driver.find_element_by_xpath('//*[@id=pairSublinksLevel2"]/li[4]/a').click()
        time.sleep(2)
        annual_cash_flow = driver.find_element_by_xpath('//*[@id="leftColumn"]/div[8]/div[1]/a[1]').click()
        time.sleep(2)
        cash_flow_table = driver.find_element_by_xpath('//*[id="excol"]').click()
        time.sleep(2)
        df_cash_flow = pd.read_html(driver.page_source)[0].dropna()
        df_cash_flow.columns = df_cash_flow.columns.droplevel(level = 1)
        time.sleep(2)


        #dataframe
        df = df_income.append(df_balance, ignore_index=True)
        df = df.appennd(df_cash_flow, ignore_index = True)
        df = df.set_index('Period Ending:')
        df.index.name = None
        df.columns = ['2021','2020']
        stacked_df = pd.DataFrame(df.stack())
        index = stacked_df.index.to_flat_index()
        stacked_df.index = index
        stacked_df = stacked_df.T
        company_name = driver.find_element_by_xpath('//*[id="leftColumn"]/div[1]/h1').text
        name_df = pd.DataFrame([{'Company' : company_name}])
        company_df = pd.concat([name_df, stacked_df], axis = 1)
        result_df = result_df.append(company_df, ignore_index = True)

        #close company info page
        driver.close

    except : 
        print('fuck')